## Preprocessing

In [26]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.2 MB/s eta 0:00:00


In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df

KeyError: ignored

In [5]:
# Determine the number of unique values in each column.
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
list(application_df['APPLICATION_TYPE'].value_counts().index)[8:]

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_df['APPLICATION_TYPE'].value_counts().index)[8:]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts = application_df['CLASSIFICATION'].value_counts().loc[lambda x: x > 1]


In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_df['CLASSIFICATION'].value_counts().index)[5:]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)
dummies_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [14]:
# Split our preprocessed data into our features and target arrays
y = dummies_df[['IS_SUCCESSFUL']]
X = dummies_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
dense1 = tf.keras.layers.Dense(input_dim = X_train_scaled.shape[1], units=80, activation = 'relu')
nn.add(dense1)
# Second hidden layer
dense2 = tf.keras.layers.Dense(units=30, activation = 'relu')
nn.add(dense2)

# Output layer
dense3 = tf.keras.layers.Dense(units=1, activation = 'sigmoid')
nn.add(dense3)
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss='MSE', optimizer = 'adam', metrics=['accuracy', 'MAE'])

In [18]:
# Train the model
nn_fit = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.1944 - accuracy: 0.7171 - MAE: 0.3874
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.1880 - accuracy: 0.7267 - MAE: 0.3748
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.1865 - accuracy: 0.7296 - MAE: 0.3719
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.1861 - accuracy: 0.7292 - MAE: 0.3708
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.1859 - accuracy: 0.7299 - MAE: 0.3705
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.1856 - accuracy: 0.7299 - MAE: 0.3697
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.1852 - accuracy: 0.7306 - MAE: 0.3695
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.1849 - accuracy: 0.7303 - MAE: 0.3689
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.184

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy, model_mae = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, MAE: {model_mae}")

268/268 - 1s - loss: 0.1837 - accuracy: 0.7335 - MAE: 0.3639 - 522ms/epoch - 2ms/step
Loss: 0.18367734551429749, Accuracy: 0.7335277199745178, MAE: 0.36385539174079895


In [20]:
# Export our model to HDF5 file
nn.save('first_model.h5')

In [30]:
#Write function to run through multiple sequential models with different hyperparamaters

def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh', 'selu'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=15,
        max_value=85,
        step=10), activation=activation, input_dim=X_train_scaled.shape[1]))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=15,
            max_value=65,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy", 'mae'])
    
    return nn_model

In [31]:
# Import the kerastuner library
import keras_tuner as kt

#create hyperband with model creation function
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [33]:
#run tuner search for best combination of hyperparameters

tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test), callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=3)])

Trial 56 Complete [00h 00m 18s]
val_accuracy: 0.7345772385597229

Best val_accuracy So Far: 0.7372594475746155
Total elapsed time: 00h 23m 08s


In [34]:
#evaluate top 3 hyperparameters

best_3 = tuner.get_best_hyperparameters(3)
for x in best_3:
    print(x.values)

{'activation': 'relu', 'first_units': 45, 'num_layers': 4, 'units_0': 55, 'units_1': 60, 'units_2': 20, 'units_3': 25, 'units_4': 30, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
{'activation': 'relu', 'first_units': 15, 'num_layers': 3, 'units_0': 15, 'units_1': 65, 'units_2': 20, 'units_3': 65, 'units_4': 35, 'tuner/epochs': 7, 'tuner/initial_epoch': 3, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0033'}
{'activation': 'relu', 'first_units': 55, 'num_layers': 5, 'units_0': 50, 'units_1': 30, 'units_2': 30, 'units_3': 15, 'units_4': 20, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}


In [35]:
#look at loss/accuracy/mae for top 3 models
best_models = tuner.get_best_models(3)
for x in best_models:
    model_loss, model_accuracy, model_mae = x.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}, MAE: {model_mae}")

268/268 - 1s - loss: 0.5486 - accuracy: 0.7373 - mae: 0.3614 - 975ms/epoch - 4ms/step
Loss: 0.5485566854476929, Accuracy: 0.7372594475746155, MAE: 0.3614128530025482
268/268 - 1s - loss: 0.5455 - accuracy: 0.7370 - mae: 0.3660 - 930ms/epoch - 3ms/step
Loss: 0.54551762342453, Accuracy: 0.7370262145996094, MAE: 0.36599257588386536
268/268 - 1s - loss: 0.5450 - accuracy: 0.7370 - mae: 0.3643 - 945ms/epoch - 4ms/step
Loss: 0.544998824596405, Accuracy: 0.7370262145996094, MAE: 0.36431819200515747
